In [30]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [31]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [32]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [33]:
model_tag = 'cohere'  
dataset_tag = '20_newsgroups'
prompt = 'random_samples'
temp = 0
number_of_experimet = 1
selected_samples =  f'{model_tag}_random24_40'

In [34]:
model = get_model_by_tag(model_tag, token, temp)  
dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
prompt_txt = get_prompt_template(dataset_tag, prompt)
examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)


In [35]:
dataset_for_annotation.columns

Index(['label', 'text'], dtype='object')

In [36]:
print(model, dataset_for_annotation, prompt_txt, examples_for_prompt, output_path)

CohereLLM
Params: {}                          label  \
0           talk.politics.misc   
1     comp.sys.ibm.pc.hardware   
2                      sci.med   
3           rec.sport.baseball   
4              sci.electronics   
...                        ...   
1995                 sci.crypt   
1996           sci.electronics   
1997           rec.motorcycles   
1998                 sci.space   
1999                 sci.crypt   

                                                   text  
0     article ashish arora writes excerpts netnewssc...  
1     gateway telepath modem month actually one woul...  
2     anybody provide advice concerning following tw...  
3     article mike silverman writes anybody know goi...  
4     article stich christian e writes installed mot...  
...                                                 ...  
1995  david sternlight writes article karl barrus wr...  
1996  hello im looking information alphanumeric page...  
1997  john r daker writes would like offocially 

In [37]:
text_col = "text"
label_col = "label"
output_col = "output"

In [38]:
annotator = LLMAnnotator(
    model=model,
    dataset=dataset_for_annotation,
    examples_for_prompt=examples_for_prompt,
    prompt_template=prompt_txt,
    column_text=text_col,
    column_label=label_col,
    column_output=output_col
)


In [39]:
df = annotator.get_results()

Nr: 0 Predicted label: talk.politics.misc
Nr: 1 Predicted label: talk.politics.misc
Nr: 2 Predicted label: sci.med
Nr: 3 Predicted label: rec.sport.baseball
Nr: 4 Predicted label: comp.sys.ibm.pc.hardware
Nr: 5 Predicted label: sci.electronics
Nr: 6 Predicted label: rec.sport.baseball
Nr: 7 Predicted label: talk.politics.misc
Nr: 8 Predicted label: talk.religion.misc
Nr: 9 Predicted label: sci.med
Nr: 10 Predicted label: talk.religion.misc
Nr: 11 Predicted label: talk.politics.misc
Nr: 12 Predicted label: talk.politics.guns
Nr: 13 Predicted label: comp.windows.x
Nr: 14 Predicted label: comp.sys.ibm.pc.hardware
Nr: 15 Predicted label: comp.os.ms-windows.misc
Nr: 16 Predicted label: rec.autos
Nr: 17 Predicted label: comp.graphics
Nr: 18 Predicted label: talk.politics.guns
Nr: 19 Predicted label: talk.politics.misc
Nr: 20 Predicted label: misc.forsale
Nr: 21 Predicted label: talk.politics.guns
Nr: 22 Predicted label: talk.religion.misc
Nr: 23 Predicted label: rec.sport.hockey
Nr: 24 Predi

In [40]:
annotator.save_results(output_path=output_path)

Results saved successfully to ./results/random_samples_temp0/cohere/20_newsgroups_cohere_random_samples_cohere_random24_40_temp0_exp1.csv


In [41]:
df = pd.read_csv(output_path)
df

,text,output,logprobs,top_logprobs,original_label
0,article ashish arora writes excerpts netnewssc...,talk.politics.misc,"{'content': [{'token': 'talk', 'bytes': [116, ...","[{'token': 'talk', 'bytes': [116, 97, 108, 107...",talk.politics.misc
1,gateway telepath modem month actually one woul...,talk.politics.misc,"{'content': [{'token': 'talk', 'bytes': [116, ...","[{'token': 'talk', 'bytes': [116, 97, 108, 107...",comp.sys.ibm.pc.hardware
2,anybody provide advice concerning following tw...,sci.med,"{'content': [{'token': 'sci', 'bytes': [115, 9...","[{'token': 'sci', 'bytes': [115, 99, 105], 'lo...",sci.med
3,article mike silverman writes anybody know goi...,rec.sport.baseball,"{'content': [{'token': 'rec', 'bytes': [114, 1...","[{'token': 'rec', 'bytes': [114, 101, 99], 'lo...",rec.sport.baseball
4,article stich christian e writes installed mot...,comp.sys.ibm.pc.hardware,"{'content': [{'token': 'comp', 'bytes': [99, 1...","[{'token': 'comp', 'bytes': [99, 111, 109, 112...",sci.electronics
...,...,...,...,...,...
1995,david sternlight writes article karl barrus wr...,talk.politics.misc,"{'content': [{'token': 'talk', 'bytes': [116, ...","[{'token': 'talk', 'bytes': [116, 97, 108, 107...",sci.crypt
1996,hello im looking information alphanumeric page...,comp.sys.ibm.pc.hardware,"{'content': [{'token': 'comp', 'bytes': [99, 1...","[{'token': 'comp', 'bytes': [99, 111, 109, 112...",sci.electronics
1997,john r daker writes would like offocially nomi...,comp.os.ms-windows.misc,"{'content': [{'token': 'comp', 'bytes': [99, 1...","[{'token': 'comp', 'bytes': [99, 111, 109, 112...",rec.motorcycles
1998,article writes looking information concerning ...,sci.space,"{'content': [{'token': 'sci', 'bytes': [115, 9...","[{'token': 'sci', 'bytes': [115, 99, 105], 'lo...",sci.space


In [42]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df.to_csv(path, index=False)
    
    return df


In [43]:
df_new = validate_text_column(output_path, examples_for_prompt)
df_new

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples
0,article ashish arora writes excerpts netnewssc...,talk.politics.misc,"{'content': [{'token': 'talk', 'bytes': [116, ...","[{'token': 'talk', 'bytes': [116, 97, 108, 107...",talk.politics.misc,False
1,gateway telepath modem month actually one woul...,talk.politics.misc,"{'content': [{'token': 'talk', 'bytes': [116, ...","[{'token': 'talk', 'bytes': [116, 97, 108, 107...",comp.sys.ibm.pc.hardware,False
2,anybody provide advice concerning following tw...,sci.med,"{'content': [{'token': 'sci', 'bytes': [115, 9...","[{'token': 'sci', 'bytes': [115, 99, 105], 'lo...",sci.med,False
3,article mike silverman writes anybody know goi...,rec.sport.baseball,"{'content': [{'token': 'rec', 'bytes': [114, 1...","[{'token': 'rec', 'bytes': [114, 101, 99], 'lo...",rec.sport.baseball,False
4,article stich christian e writes installed mot...,comp.sys.ibm.pc.hardware,"{'content': [{'token': 'comp', 'bytes': [99, 1...","[{'token': 'comp', 'bytes': [99, 111, 109, 112...",sci.electronics,False
...,...,...,...,...,...,...
1995,david sternlight writes article karl barrus wr...,talk.politics.misc,"{'content': [{'token': 'talk', 'bytes': [116, ...","[{'token': 'talk', 'bytes': [116, 97, 108, 107...",sci.crypt,False
1996,hello im looking information alphanumeric page...,comp.sys.ibm.pc.hardware,"{'content': [{'token': 'comp', 'bytes': [99, 1...","[{'token': 'comp', 'bytes': [99, 111, 109, 112...",sci.electronics,False
1997,john r daker writes would like offocially nomi...,comp.os.ms-windows.misc,"{'content': [{'token': 'comp', 'bytes': [99, 1...","[{'token': 'comp', 'bytes': [99, 111, 109, 112...",rec.motorcycles,False
1998,article writes looking information concerning ...,sci.space,"{'content': [{'token': 'sci', 'bytes': [115, 9...","[{'token': 'sci', 'bytes': [115, 99, 105], 'lo...",sci.space,False
